# Floater Study Notebook

This notebook implements a simple hydrostatic and eigenperiod model for MOLO-type floaters.
Edit the configuration cell to explore different geometries and mass distributions.


In [1]:
import math
from dataclasses import dataclass
from typing import List, Optional

G = 9.81
RHO_W = 1025.0

@dataclass
class Column:
    radius: float
    diameter: float
    draft: float
    freeboard: float

@dataclass
class MassItem:
    mass: float
    z: float

@dataclass
class LowerPlates:
    n_plates: int
    length: float
    width: float

@dataclass
class FloaterConfig:
    columns: List[Column]
    lower_plates: LowerPlates
    mass_items: List[MassItem]
    water_density: float = RHO_W
    gravity: float = G
    ref_mass_total: Optional[float] = None
    ref_C33: Optional[float] = None
    ref_T_heave: Optional[float] = None
    ref_plate_length: Optional[float] = None
    ref_plate_width: Optional[float] = None


In [2]:
def compute_displacement(config: FloaterConfig) -> float:
    V = 0.0
    for col in config.columns:
        area = math.pi * (col.diameter / 2.0)**2
        V += area * col.draft
    return V

def compute_waterplane_area(config: FloaterConfig) -> float:
    A = 0.0
    for col in config.columns:
        A += math.pi * (col.diameter / 2.0)**2
    return A

def compute_zB(config: FloaterConfig) -> float:
    drafts = [c.draft for c in config.columns]
    draft_mean = sum(drafts) / len(drafts)
    return -draft_mean / 2.0

def compute_zG(config: FloaterConfig) -> float:
    m_total = sum(m.mass for m in config.mass_items)
    if m_total == 0.0:
        return 0.0
    zG = sum(m.mass * m.z for m in config.mass_items) / m_total
    return zG

def compute_pitch_BM(config: FloaterConfig, disp_volume: float) -> float:
    I_wp = 0.0
    for col in config.columns:
        A = math.pi * (col.diameter / 2.0)**2
        I_wp += A * (col.radius**2) / 2.0
    BM = I_wp / disp_volume
    return BM

def heave_stiffness(config: FloaterConfig, Aw: float) -> float:
    return config.water_density * config.gravity * Aw

def pitch_stiffness(config: FloaterConfig, disp_volume: float, GM: float) -> float:
    return config.water_density * config.gravity * disp_volume * GM

def calibrate_heave_added_mass_ratio(config: FloaterConfig) -> float:
    if config.ref_mass_total is None or config.ref_C33 is None or config.ref_T_heave is None:
        return 4.0
    m = config.ref_mass_total
    C33 = config.ref_C33
    T = config.ref_T_heave
    m_eff = (T / (2.0 * math.pi))**2 * C33
    a33 = m_eff - m
    return a33 / m

def scale_heave_added_mass_ratio(config: FloaterConfig, ref_ratio: float) -> float:
    if config.ref_plate_length is None or config.ref_plate_width is None:
        return ref_ratio
    A_ref = config.ref_plate_length * config.ref_plate_width * 3.0
    plates = config.lower_plates
    A_new = plates.length * plates.width * plates.n_plates
    return ref_ratio * (A_new / A_ref)

def heave_period(config: FloaterConfig, disp_volume: float, Aw: float) -> float:
    m_struct = sum(m.mass for m in config.mass_items) * 1000.0
    C33 = heave_stiffness(config, Aw)
    ref_ratio = calibrate_heave_added_mass_ratio(config)
    ratio_new = scale_heave_added_mass_ratio(config, ref_ratio)
    m_eff = m_struct * (1.0 + ratio_new)
    return 2.0 * math.pi * math.sqrt(m_eff / C33)

def pitch_period(config: FloaterConfig, GM: float) -> float:
    I_struct = 0.0
    for m in config.mass_items:
        I_struct += m.mass * (m.z**2)
    I_struct *= 1000.0
    V = compute_displacement(config)
    C_theta = pitch_stiffness(config, V, GM)
    return 2.0 * math.pi * math.sqrt(I_struct / C_theta)

def summarize_floater(config: FloaterConfig) -> None:
    V = compute_displacement(config)
    m_disp = V * config.water_density / 1000.0
    Aw = compute_waterplane_area(config)
    zB = compute_zB(config)
    zG = compute_zG(config)
    BG = zG - zB
    BM = compute_pitch_BM(config, V)
    GM = BM - BG
    C33 = heave_stiffness(config, Aw)
    Ctheta = pitch_stiffness(config, V, GM)
    T_heave = heave_period(config, V, Aw)
    T_pitch = pitch_period(config, GM)
    print("=== Floater summary ===")
    print(f"Displacement volume   : {V:8.1f} m³")
    print(f"Displacement mass     : {m_disp:8.1f} t")
    print(f"Waterplane area       : {Aw:8.1f} m²")
    print(f"zB (CoB)              : {zB:8.2f} m")
    print(f"zG (CoG)              : {zG:8.2f} m")
    print(f"BG                    : {BG:8.2f} m")
    print(f"BM (pitch)            : {BM:8.2f} m")
    print(f"GM (pitch)            : {GM:8.2f} m")
    print(f"C33 (heave stiff.)    : {C33:8.3e} N/m")
    print(f"Cθ  (pitch stiff.)    : {Ctheta:8.3e} Nm/rad")
    print()
    print(f"Heave period T33      : {T_heave:6.2f} s")
    print(f"Pitch/Roll period Tθ  : {T_pitch:6.2f} s")
    print("=======================")


In [3]:
# Example configuration: 24 MW, 6+1 layout
draft = 24.0
freeboard = 12.0
R_in = 50.0
R_out = 84.0

columns = []
for _ in range(3):
    columns.append(Column(radius=R_in, diameter=9.0, draft=draft, freeboard=freeboard))
    columns.append(Column(radius=R_out, diameter=9.0, draft=draft, freeboard=freeboard))
columns.append(Column(radius=0.0, diameter=10.0, draft=draft, freeboard=freeboard))

lower_plates = LowerPlates(
    n_plates=3,
    length=R_out,
    width=14.5,
)

mass_items = [
    MassItem(mass=1170.0, z=80.0),   # tower
    MassItem(mass=1350.0, z=160.0),  # RNA
    MassItem(mass=5100.0, z=-5.0),   # steel
    MassItem(mass=4000.0, z=-20.0),  # ballast
]

ref_disp_mass = 7975.0 * 1000.0
Aw_ref = 7.0 * math.pi * (10.5 / 2.0)**2
C33_ref = RHO_W * G * Aw_ref
T_heave_ref = 16.0
R_ref = 67.22
B_ref = 14.0

config = FloaterConfig(
    columns=columns,
    lower_plates=lower_plates,
    mass_items=mass_items,
    ref_mass_total=ref_disp_mass,
    ref_C33=C33_ref,
    ref_T_heave=T_heave_ref,
    ref_plate_length=R_ref,
    ref_plate_width=B_ref,
)

summarize_floater(config)


=== Floater summary ===
Displacement volume   :  11045.8 m³
Displacement mass     :  11322.0 t
Waterplane area       :    460.2 m²
zB (CoB)              :   -12.00 m
zG (CoG)              :    17.56 m
BG                    :    29.56 m
BM (pitch)            :    82.56 m
GM (pitch)            :    52.99 m
C33 (heave stiff.)    : 4.628e+06 N/m
Cθ  (pitch stiff.)    : 5.886e+09 Nm/rad

Heave period T33      :  24.63 s
Pitch/Roll period Tθ  :  17.14 s
